In [1]:
#import the libraries

import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import torch
import torch.nn as nn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\khush\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
print('pandas version = ',pd.__version__)
print('numpy version = ',np.__version__)
print('nltk version = ',nltk.__version__)
print('regex version = ',re.__version__)
#print(BeautifulSoup.__version__)
print('torch version = ',torch.__version__)

pandas version =  1.3.5
numpy version =  1.21.4
nltk version =  3.7
regex version =  2.2.1
torch version =  1.11.0+cpu


### 1.) Dataset Generation

In [30]:
 data1 = pd.read_csv('./data.tsv.gz', compression='gzip', 
                                 header=0, sep='\t', error_bad_lines=False)

C:\Users\khush\anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 10093: expected 15 fields, saw 22\nSkipping line 31965: expected 15 fields, saw 22\nSkipping line 49886: expected 15 fields, saw 22\nSkipping line 49905: expected 15 fields, saw 22\n'
b'Skipping line 67579: expected 15 fields, saw 22\nSkipping line 75367: expected 15 fields, saw 22\nSkipping line 92462: expected 15 fields, saw 22\nSkipping line 105041: expected 15 fields, saw 22\nSkipping line 109697: expected 15 fields, saw 22\nSkipping line 121931: expected 15 fields, saw 22\n'
b'Skipping line 139492: expected 15 fields, saw 22\nSkipping line 158729: expected 15 fields, saw 22\nSkipping line 165784: expected 15 fields, saw 22\nSkipping line 176996: expected 15 fields, saw 22\nSkipping line 182928: expected 15 fields, saw 22

b'Skipping line 1979093: expected 15 fields, saw 22\nSkipping line 1982997: expected 15 fields, saw 22\nSkipping line 1992924: expected 15 fields, saw 22\nSkipping line 1996161: expected 15 fields, saw 22\nSkipping line 2003175: expected 15 fields, saw 22\nSkipping line 2024153: expected 15 fields, saw 22\nSkipping line 2026345: expected 15 fields, saw 22\n'
b'Skipping line 2041159: expected 15 fields, saw 22\nSkipping line 2042954: expected 15 fields, saw 22\nSkipping line 2044244: expected 15 fields, saw 22\nSkipping line 2047949: expected 15 fields, saw 22\nSkipping line 2051022: expected 15 fields, saw 22\nSkipping line 2052365: expected 15 fields, saw 22\nSkipping line 2064460: expected 15 fields, saw 22\nSkipping line 2077010: expected 15 fields, saw 22\nSkipping line 2083893: expected 15 fields, saw 22\n'
b'Skipping line 2097514: expected 15 fields, saw 22\nSkipping line 2100479: expected 15 fields, saw 22\nSkipping line 2103183: expected 15 fields, saw 22\nSkipping line 2108608

In [31]:
 data = data1[['star_rating','review_body']]
 data.tail()

,star_rating,review_body
5094302,5,After watching my Dad struggle with his scisso...
5094303,3,"Like most sound machines, the sounds choices a..."
5094304,5,I bought this product because it indicated 30 ...
5094305,5,We have used Oral-B products for 15 years; thi...
5094306,5,"I love this toothbrush. It's easy to use, and ..."


In [4]:
data = data.dropna()

In [5]:
data['star_rating'] = data['star_rating'].astype(str).astype(float).astype(int)
data['star_rating'].dtype

dtype('int32')

In [6]:
data['star_rating'] = data['star_rating'].apply(lambda x: 2 if x==3 else( 1 if 2>=x else 3 ))

In [7]:
data['star_rating'].value_counts()

3    3979156
1     717991
2     396760
Name: star_rating, dtype: int64

In [8]:
import random
# Return a list of indices
pos_idx = random.sample(list(data.query('star_rating==3').index),20000)
neg_idx = random.sample(list(data.query('star_rating==2').index),20000)
neu_idx = random.sample(list(data.query('star_rating==1').index),20000)

# Use indices to select data
df_sample = data.loc[pos_idx + neg_idx + neu_idx]

In [9]:
df_sample['star_rating'].value_counts()

3    20000
2    20000
1    20000
Name: star_rating, dtype: int64

In [10]:
df_sample.head()

,star_rating,review_body
4597883,3,"It felt thick when I first put it on, but it d..."
3887234,3,"OK, not sure why I didn't get these sooner. My..."
3895493,3,"I heard my hairdress talking about Argan oil, ..."
863075,3,I love it!!!! Wow first time my daughters no c...
12788,3,Really like how it holds without being so stif...


### 2) Word Embeddings

#### a) Load pre-trained models

In [11]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path = './GoogleNews-vectors-negative300.bin.gz'
w2v_model = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [26]:
print('excellent and outstanding similarity =',w2v_model.similarity('excellent','outstanding'))

excellent and outstanding similarity = 0.5567486


In [12]:
print('dog and cat similarity =',w2v_model.similarity('dog','cat'))

dog and cat similarity = 0.76094574


In [24]:
print('man is most similar =',w2v_model.most_similar(positive=['man'], topn=1))

man is most similar = [('woman', 0.7664011716842651)]


#### b) Train Word2Vec Model

In [16]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
sent = []
#print(sent_tokenize('This is. An example.'))
#print(sent_tokenize(df_sample['review_body'][0]))
sent.append(df_sample['review_body'].apply(lambda x: sent_tokenize(x)))
flat_list = [item for sublist in sent for item in sublist]
sentences = [item.split() for sublist in flat_list for item in sublist]
print(type(sentences))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<class 'list'>


In [17]:
print(sentences[:5])
sen = ['This is.' ,'An example.','hello how are you.']

[['It', 'felt', 'thick', 'when', 'I', 'first', 'put', 'it', 'on,', 'but', 'it', 'disappeared', 'into', 'the', 'skin', 'and', 'it', 'actually', 'makes', 'your', 'face', 'feel', 'like', 'it', 'is', 'tightened?'], ['But', 'not', 'in', 'a', 'bad', 'way,', 'it', 'feels', 'like', 'you', 'have', 'some', 'of', 'your', 'youth', 'back.'], ['Skin', 'is', 'silky', 'next', 'morning.'], ['Would', 'buy', 'again,', 'promplty', 'shipped.'], ['OK,', 'not', 'sure', 'why', 'I', "didn't", 'get', 'these', 'sooner.']]


In [18]:
from gensim.models import Word2Vec

#sentences = MyCorpus()
#print(sentences)
#model.build_vocab(sen, update=True)
model = Word2Vec(sentences, min_count = 9, vector_size = 300, window=13)
model.build_vocab(sentences, update=True)
print(model)

model.train(sentences, total_examples=model.corpus_count, epochs=10)

Word2Vec(vocab=13261, vector_size=300, alpha=0.025)


(21486833, 30450170)

In [25]:
print('excellent and outstanding similarity =',model.wv.similarity('excellent','outstanding'))

excellent and outstanding similarity = 0.65382564


In [20]:
print('dog and cat similarity =',model.wv.similarity('dog','cat'))

dog and cat similarity = 0.46736932


In [23]:
print('man is most similar =',model.wv.most_similar(positive=['man'], topn=1))

man is most similar = [('woman', 0.6469601988792419)]


### 3. Simple Models

In [19]:
#total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)
df_sample['review_body'] = df_sample['review_body'].str.lower()

In [20]:
df_sample['review_body'] = df_sample['review_body'].replace(r'http\S+', ' ', regex=True).replace(r'www\S+', ' ', regex=True)

In [21]:
df_sample['review_body'] = df_sample.review_body.str.replace('[^a-zA-Z]', ' ')

C:\Users\khush\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [22]:
df_sample.review_body = df_sample.review_body.replace(r'\s+', ' ', regex=True)

In [23]:
!pip install contractions
import contractions

In [24]:
#rws["expanded_words"]=rws['comments'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df_sample['review_body'] = df_sample['review_body'].astype(str)
df_sample['review_body_expanded'] = df_sample['review_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [25]:
df_sample['review_body_expanded'] = df_sample['review_body_expanded'].apply(lambda x: ' '.join(x))
df_sample.tail()

,star_rating,review_body,review_body_expanded
4291501,1,the color pictures is not the color i received...,the color pictures is not the color i received...
2431618,1,it burns my face its really unpleasant i now u...,it burns my face its really unpleasant i now u...
3826128,1,if you like hair gel than go with this product...,if you like hair gel than go with this product...
4807272,1,i was very curious about this scent especially...,i was very curious about this scent especially...
2192929,1,it made a terrible voice and did not cut,it made a terrible voice and did not cut


In [26]:
df_sample['star_rating'].value_counts()

3    20000
2    20000
1    20000
Name: star_rating, dtype: int64

In [27]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words('english')
#test['tweet_without_stopwords'] = test['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_sample['review_body_swr'] = df_sample['review_body_expanded'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_sample.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,star_rating,review_body,review_body_expanded,review_body_swr
1689352,3,great light weight i just find it easier to st...,great light weight i just find it easier to st...,great light weight find easier style hair
3928830,3,i ve used this product before it s excellent i...,i ve used this product before it s excellent i...,used product excellent much easier order line ...
515546,3,great product,great product,great product
4486603,3,our kids love the taste of this toothpaste and...,our kids love the taste of this toothpaste and...,kids love taste toothpaste good job far cleani...
2006570,3,does wonders to dry skin would recommend,does wonders to dry skin would recommend,wonders dry skin would recommend


In [28]:
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
# lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

df_sample['text_lemmatized'] = df_sample.review_body_expanded.apply(lemmatize_text)
df_sample.head()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,star_rating,review_body,review_body_expanded,review_body_swr,text_lemmatized
1689352,3,great light weight i just find it easier to st...,great light weight i just find it easier to st...,great light weight find easier style hair,great light weight i just find it easier to st...
3928830,3,i ve used this product before it s excellent i...,i ve used this product before it s excellent i...,used product excellent much easier order line ...,i ve used this product before it s excellent i...
515546,3,great product,great product,great product,great product
4486603,3,our kids love the taste of this toothpaste and...,our kids love the taste of this toothpaste and...,kids love taste toothpaste good job far cleani...,our kid love the taste of this toothpaste and ...
2006570,3,does wonders to dry skin would recommend,does wonders to dry skin would recommend,wonders dry skin would recommend,doe wonder to dry skin would recommend


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
x_idf = v.fit_transform(df_sample['text_lemmatized'])
x_idf.shape

(60000, 30691)

In [30]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
sent = []
#print(sent_tokenize('This is. An example.'))
#print(sent_tokenize(df_sample['review_body'][0]))
df_sample['tokenize'] = df_sample['text_lemmatized'].apply(lambda x: x.split())
#flat_list = [item for sublist in sent for item in sublist]
#sentences = [item.split() for sublist in flat_list for item in sublist]
#print(sentences[:2])
df_sample.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,star_rating,review_body,review_body_expanded,review_body_swr,text_lemmatized,tokenize
1689352,3,great light weight i just find it easier to st...,great light weight i just find it easier to st...,great light weight find easier style hair,great light weight i just find it easier to st...,"[great, light, weight, i, just, find, it, easi..."
3928830,3,i ve used this product before it s excellent i...,i ve used this product before it s excellent i...,used product excellent much easier order line ...,i ve used this product before it s excellent i...,"[i, ve, used, this, product, before, it, s, ex..."
515546,3,great product,great product,great product,great product,"[great, product]"
4486603,3,our kids love the taste of this toothpaste and...,our kids love the taste of this toothpaste and...,kids love taste toothpaste good job far cleani...,our kid love the taste of this toothpaste and ...,"[our, kid, love, the, taste, of, this, toothpa..."
2006570,3,does wonders to dry skin would recommend,does wonders to dry skin would recommend,wonders dry skin would recommend,doe wonder to dry skin would recommend,"[doe, wonder, to, dry, skin, would, recommend]"


In [31]:
from operator import add
fans = []
for X in df_sample['tokenize']:
    res = []
    l = len(X)
    if(l==0):
        fans.append(np.zeros(300,dtype = float))
        continue
    #print(l)
    for i in X:
        if(i in w2v_model):
            res.append(w2v_model[i])
        else:
            res.append(np.zeros(300,dtype = np.float64))
    res_arr = [0] * 300
    for i in res:
        res_arr = list(map(add, res_arr, i))
    
    ans = [x / l for x in res_arr]
    fans.append(ans)

#### Accuracy for perceptron using word2vec

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        fans, 
        df_sample['star_rating'],
        train_size=0.80, 
        random_state=1234)

In [33]:
from sklearn.metrics import classification_report

from sklearn.linear_model import Perceptron
p = Perceptron(random_state=42)
p.fit(X_train, y_train)
y_pred = p.predict(X_test)

from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, labels=[1, 2, 3])
print(report)

              precision    recall  f1-score   support

           1       0.60      0.73      0.66      4018
           2       0.65      0.18      0.29      4002
           3       0.57      0.85      0.69      3980

    accuracy                           0.59     12000
   macro avg       0.61      0.59      0.54     12000
weighted avg       0.61      0.59      0.54     12000



#### Accuracy for perceptron using TFIDF

In [34]:
from sklearn.model_selection import train_test_split

X_train_idf, X_test_idf, y_train_idf, y_test_idf  = train_test_split(
        x_idf, 
        df_sample['star_rating'],
        train_size=0.80, 
        random_state=1234)

In [35]:
from sklearn.metrics import classification_report

from sklearn.linear_model import Perceptron
p = Perceptron(random_state=42)
p.fit(X_train_idf, y_train_idf)
y_pred_idf = p.predict(X_test_idf)

from sklearn.metrics import classification_report
report = classification_report(y_test_idf, y_pred_idf, labels=[1, 2, 3])
print(report)

              precision    recall  f1-score   support

           1       0.67      0.60      0.63      4018
           2       0.54      0.53      0.54      4002
           3       0.68      0.76      0.72      3980

    accuracy                           0.63     12000
   macro avg       0.63      0.63      0.63     12000
weighted avg       0.63      0.63      0.63     12000



#### Accuracy for SVM using word2vec

In [36]:
from sklearn import svm

clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

report = classification_report(y_test, y_pred, labels=[1, 2, 3])
print(report)

              precision    recall  f1-score   support

           1       0.69      0.68      0.69      4018
           2       0.59      0.62      0.60      4002
           3       0.77      0.73      0.75      3980

    accuracy                           0.68     12000
   macro avg       0.68      0.68      0.68     12000
weighted avg       0.68      0.68      0.68     12000



#### Accuracy for SVM using TFID

In [37]:
from sklearn import svm

clf = svm.SVC(kernel='rbf')
clf.fit(X_train_idf, y_train_idf)

#Predict the response for test dataset
y_pred_idf = clf.predict(X_test_idf)

report = classification_report(y_test_idf, y_pred_idf, labels=[1, 2, 3])
print(report)

              precision    recall  f1-score   support

           1       0.73      0.73      0.73      4018
           2       0.63      0.66      0.65      4002
           3       0.82      0.79      0.80      3980

    accuracy                           0.73     12000
   macro avg       0.73      0.73      0.73     12000
weighted avg       0.73      0.73      0.73     12000



### 4. Feed Forward Neural Networks

#### a) FNN on average word2vec embeddings

In [38]:
import torch

In [39]:
torch.cuda.is_available()

False

In [40]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        current_sample = self.data[idx]
        current_target = self.targets[idx]-1
        return torch.tensor(current_sample, dtype = torch.float), torch.tensor(current_target, dtype = torch.long)

In [41]:
train_dataset = CustomDataset(data = X_train, targets = y_train.values)
test_dataset = CustomDataset(data = X_test, targets = y_test.values)

In [42]:
(train_dataset[4])

(tensor([ 0.0247,  0.0128,  0.0298,  0.1084, -0.0263, -0.0027,  0.0235, -0.0897,
          0.0498,  0.0690, -0.0262, -0.0855, -0.0412, -0.0036, -0.0947,  0.0852,
          0.0553,  0.0921,  0.0011, -0.0448, -0.0338,  0.0657,  0.0569, -0.0268,
          0.0257, -0.0056, -0.0693,  0.0593, -0.0115, -0.0288, -0.0383,  0.0337,
         -0.0244, -0.0164, -0.0106, -0.0103, -0.0236, -0.0226,  0.0286,  0.0390,
          0.0770, -0.0439,  0.0872, -0.0164,  0.0029,  0.0037, -0.0082, -0.0278,
          0.0362,  0.0040, -0.0008,  0.0692, -0.0055,  0.0231,  0.0051,  0.0364,
         -0.0163, -0.0449,  0.0163, -0.0531, -0.0665,  0.0501, -0.0662, -0.0801,
          0.0032, -0.0080, -0.0597,  0.0828, -0.0285,  0.0752,  0.0556,  0.0334,
          0.0630,  0.0205, -0.1092, -0.0389,  0.0958,  0.0893,  0.0166,  0.0664,
         -0.0148, -0.0813,  0.0365,  0.0053, -0.0508, -0.0306, -0.0576,  0.1259,
         -0.0149,  0.0418,  0.0137,  0.0462, -0.0479, -0.0445, -0.0575, -0.0596,
          0.0114,  0.0245,  

In [43]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
    num_workers=0)

In [44]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(300,100)
        self.fc2 = nn.Linear(100,10)
        self.fc3 = nn.Linear(10,3)
        self.softmax = nn.LogSoftmax(dim=1)
            
    def forward(self, features):
        x = F.relu(self.fc1(features))
        x = F.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [45]:
model = Model()
print(model)

Model(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [47]:
for epoch in range(7):
    train_loss = 0.0
    model.train()
    for data, target in train_loader:
        #print(type(data))
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss))
        

Epoch: 1 	Training Loss: 0.824479
Epoch: 2 	Training Loss: 0.775426
Epoch: 3 	Training Loss: 0.761249
Epoch: 4 	Training Loss: 0.751303
Epoch: 5 	Training Loss: 0.743251
Epoch: 6 	Training Loss: 0.735158
Epoch: 7 	Training Loss: 0.728139


In [48]:
def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        print()
        outputs = model(batch)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
    return prediction_list

In [49]:
num_correct = 0
total = 0
for data, labels in test_loader:
            predictions = model(data)
            print(predictions)
            #print(torch.argmax(predictions)==labels)
            for i in range(len(predictions)):
                if(torch.argmax(predictions[i]) == labels[i]):
                    num_correct += 1

            #print(torch.argmax(predictions[0]))
            #print(labels[0])
            #num_correct += (torch.argmax(predictions)==labels).sum()
            total += labels.size(0)

tensor([[-1.4514, -0.9906, -0.9304],
        [-0.2914, -1.5280, -3.3301],
        [-1.4932, -1.0874, -0.8250],
        [-1.7777, -0.6928, -1.1063],
        [-3.6126, -2.2696, -0.1396],
        [-1.4418, -0.4363, -2.1449],
        [-0.6491, -0.9503, -2.3985],
        [-2.0524, -0.2629, -2.2756],
        [-2.0987, -0.2285, -2.5050],
        [-4.3727, -3.5385, -0.0426],
        [-3.9211, -3.2041, -0.0623],
        [-2.2664, -0.2359, -2.2399],
        [-2.7880, -0.7570, -0.7564],
        [-1.9947, -0.6672, -1.0475],
        [-1.4599, -0.4959, -1.8407],
        [-1.7262, -0.3886, -1.9376],
        [-1.1630, -0.4332, -3.2438],
        [-0.9634, -0.6568, -2.3036],
        [-0.6253, -0.8301, -3.5435],
        [-1.9813, -0.4010, -1.6477],
        [-5.4896, -4.2543, -0.0185],
        [-1.8004, -0.4253, -1.7081],
        [-1.3798, -1.5283, -0.6321],
        [-3.5976, -3.0600, -0.0772],
        [-1.6919, -1.1683, -0.6833],
        [-0.2859, -1.6834, -2.7664],
        [-2.5907, -1.7033, -0.2971],
 

tensor([[-3.2323, -2.0906, -0.1780],
        [-0.8234, -1.3384, -1.2080],
        [-0.6112, -1.0848, -2.1258],
        [-2.6629, -1.3372, -0.4040],
        [-0.5982, -1.0441, -2.3207],
        [-1.3394, -0.5644, -1.7762],
        [-1.5732, -0.3533, -2.4051],
        [-2.3258, -0.7944, -0.7975],
        [-0.3947, -1.5548, -2.1638],
        [-1.1613, -1.3155, -0.8709],
        [-1.5589, -0.4505, -1.8819],
        [-2.0219, -0.3686, -1.7381],
        [-1.9358, -0.4435, -1.5423],
        [-1.4173, -1.2926, -0.7275],
        [-2.4976, -0.9824, -0.6101],
        [-1.7223, -0.9307, -0.8508],
        [-0.6985, -0.9129, -2.2896],
        [-3.3690, -2.3653, -0.1374],
        [-1.2918, -0.4309, -2.5862],
        [-3.4442, -2.4293, -0.1279],
        [-0.7107, -1.0123, -1.9287],
        [-4.6473, -2.0958, -0.1422],
        [-1.6991, -1.0634, -0.7511],
        [-0.7753, -1.0071, -1.7478],
        [-0.3950, -1.5543, -2.1630],
        [-0.9485, -0.6514, -2.3932],
        [-2.3830, -0.7337, -0.8496],
 

tensor([[-5.3097, -4.0945, -0.0218],
        [-0.4437, -1.4790, -2.0365],
        [-1.6680, -0.6975, -1.1598],
        [-2.4779, -1.2795, -0.4496],
        [-1.6866, -0.4407, -1.7645],
        [-1.0567, -1.0572, -1.1875],
        [-0.8513, -0.7946, -2.1087],
        [-0.2333, -1.9313, -2.7624],
        [-1.4200, -0.6669, -1.4065],
        [-2.0083, -1.5098, -0.4388],
        [-0.2879, -1.7442, -2.5854],
        [-0.2575, -1.6739, -3.2306],
        [-1.5861, -0.8367, -1.0157],
        [-2.0455, -0.3515, -1.7895],
        [-0.3884, -1.4998, -2.3156],
        [-2.1679, -1.3996, -0.4480],
        [-0.3198, -1.3795, -3.8164],
        [-4.6394, -3.4996, -0.0407],
        [-1.9368, -0.4425, -1.5446],
        [-2.0756, -0.8116, -0.8431],
        [-0.8879, -1.0927, -1.3738],
        [-3.2175, -2.2161, -0.1615],
        [-0.2907, -1.8708, -2.3199],
        [-4.1164, -3.0398, -0.0663],
        [-3.9575, -2.8964, -0.0772],
        [-1.6499, -0.8830, -0.9304],
        [-4.7185, -3.5698, -0.0378],
 

tensor([[-1.3650, -1.1127, -0.8772],
        [-0.3766, -1.4520, -2.5291],
        [-0.6723, -1.1625, -1.7329],
        [-2.1824, -1.6799, -0.3555],
        [-2.1524, -0.2874, -2.0131],
        [-5.3211, -4.1046, -0.0216],
        [-1.0493, -0.5168, -2.9303],
        [-0.0764, -2.8557, -4.1333],
        [-1.7441, -0.4952, -1.5337],
        [-1.0754, -0.7824, -1.6018],
        [-0.4764, -1.0701, -3.3250],
        [-5.6398, -4.3878, -0.0161],
        [-0.9640, -1.5662, -0.8921],
        [-1.4414, -1.0259, -0.9041],
        [-1.9032, -1.1285, -0.6398],
        [-3.4415, -2.5948, -0.1128],
        [-0.0593, -3.0887, -4.4202],
        [-0.9370, -1.6091, -0.8961],
        [-2.4048, -1.4081, -0.4078],
        [-1.1984, -0.6280, -1.8038],
        [-1.9260, -1.6523, -0.4115],
        [-1.7175, -0.9816, -0.8079],
        [-6.1107, -4.8065, -0.0104],
        [-1.2677, -0.4996, -2.1916],
        [-2.8472, -1.8752, -0.2374],
        [-2.1299, -0.7369, -0.9099],
        [-1.4317, -0.5427, -1.7154],
 

tensor([[-0.9406, -0.7005, -2.1778],
        [-0.4993, -1.4226, -1.8842],
        [-2.2985, -0.2237, -2.3024],
        [-3.3592, -2.3702, -0.1372],
        [-2.1632, -0.3934, -1.5594],
        [-1.1204, -0.7421, -1.6208],
        [-0.0113, -4.6099, -6.6384],
        [-1.3057, -0.6645, -1.5395],
        [-0.5916, -1.3128, -1.7289],
        [-3.3471, -2.3596, -0.1389],
        [-0.2532, -1.8693, -2.6666],
        [-5.2789, -4.0671, -0.0225],
        [-1.2096, -0.4271, -3.0105],
        [-0.7042, -1.0529, -1.8541],
        [-0.9242, -0.7621, -1.9916],
        [-2.0511, -1.2981, -0.5135],
        [-1.7153, -0.4061, -1.8718],
        [-2.7136, -0.8911, -0.6473],
        [-1.4455, -0.8690, -1.0643],
        [-3.6579, -2.1464, -0.1540],
        [-0.4203, -1.4337, -2.2567],
        [-3.4378, -2.9888, -0.0861],
        [-0.8289, -0.7272, -2.5233],
        [-0.5741, -1.0976, -2.2717],
        [-4.6796, -3.5353, -0.0392],
        [-0.2644, -2.2113, -2.0976],
        [-0.0729, -2.8643, -4.3241],
 

...

tensor([[-0.4695, -1.3813, -2.0921],
        [-0.7512, -0.9267, -2.0224],
        [-3.0154, -1.7290, -0.2568],
        [-1.2638, -0.8800, -1.1952],
        [-4.3668, -3.2582, -0.0525],
        [-0.0760, -2.8605, -4.1401],
        [-1.4349, -0.5549, -1.6729],
        [-0.4817, -1.2758, -2.2727],
        [-1.9217, -1.1435, -0.6256],
        [-2.1878, -0.2699, -2.0846],
        [-1.6125, -0.9024, -0.9289],
        [-1.9447, -0.4346, -1.5632],
        [-1.9072, -0.4744, -1.4730],
        [-1.9676, -1.5873, -0.4220],
        [-0.0683, -2.9543, -4.2759],
        [-1.0970, -0.7585, -1.6207],
        [-4.6219, -3.4842, -0.0414],
        [-0.7225, -0.9936, -1.9366],
        [-3.4115, -3.0018, -0.0863],
        [-1.9680, -0.4133, -1.6155],
        [-2.0762, -0.8618, -0.7936],
        [-3.0332, -2.0856, -0.1892],
        [-2.0156, -0.5996, -1.1465],
        [-2.1564, -0.5945, -1.1014],
        [-1.7078, -0.6397, -1.2333],
        [-4.2745, -3.1765, -0.0573],
        [-5.6784, -4.4222, -0.0155],
 

tensor([[-1.9935e-01, -2.0701e+00, -2.9083e+00],
        [-1.4971e+00, -9.8741e-01, -9.0710e-01],
        [-2.1962e+00, -2.6594e-01, -2.1014e+00],
        [-1.8370e+00, -9.6720e-01, -7.7530e-01],
        [-8.9678e-01, -6.6936e-01, -2.5247e+00],
        [-3.8011e-02, -3.4810e+00, -5.0326e+00],
        [-3.1939e+00, -1.7513e+00, -2.4151e-01],
        [-5.5759e-01, -1.1280e+00, -2.2660e+00],
        [-3.7575e+00, -2.7201e+00, -9.3442e-02],
        [-4.5188e+00, -3.2472e+00, -5.1066e-02],
        [-1.7438e+00, -8.0535e-01, -9.7232e-01],
        [-3.3135e-01, -1.6741e+00, -2.3585e+00],
        [-1.5182e-01, -2.2718e+00, -3.2772e+00],
        [-9.1693e-01, -1.4009e+00, -1.0388e+00],
        [-5.4653e-01, -1.3613e+00, -1.8035e+00],
        [-4.1809e-01, -1.2056e+00, -3.1656e+00],
        [-2.0163e+00, -3.3993e-01, -1.8641e+00],
        [-2.8777e+00, -1.2315e+00, -4.2791e-01],
        [-5.4873e+00, -4.2523e+00, -1.8542e-02],
        [-2.5882e+00, -1.2912e+00, -4.3095e-01],
        [-1.5704e+00

tensor([[-4.1478, -3.1054, -0.0625],
        [-2.0214, -0.7557, -0.9216],
        [-3.3825, -2.2179, -0.1541],
        [-2.5946, -1.2015, -0.4707],
        [-0.1428, -2.3224, -3.3524],
        [-2.4089, -1.1316, -0.5317],
        [-1.4500, -0.4151, -2.2524],
        [-0.1968, -2.0549, -2.9845],
        [-0.1267, -2.4216, -3.4991],
        [-0.8834, -1.4272, -1.0595],
        [-1.7951, -0.7995, -0.9562],
        [-1.3786, -1.7622, -0.5509],
        [-0.7451, -0.9729, -1.9152],
        [-2.3493, -2.0030, -0.2618],
        [-5.9889, -4.6982, -0.0117],
        [-1.6444, -0.4238, -1.8817],
        [-0.1699, -2.0101, -3.8037],
        [-0.1402, -2.3376, -3.3748],
        [-2.8469, -2.1982, -0.1852],
        [-0.4873, -1.0771, -3.0975],
        [-0.4210, -1.5126, -2.0933],
        [-0.8453, -1.0701, -1.4800],
        [-3.1411, -2.1795, -0.1700],
        [-2.6358, -1.4159, -0.3774],
        [-2.1688, -0.9627, -0.6855],
        [-0.8883, -0.7029, -2.3697],
        [-1.5002, -0.5113, -1.7306],
 

tensor([[-1.1321, -0.9003, -1.3049],
        [-0.7106, -0.7949, -2.8650],
        [-4.5595, -3.4288, -0.0438],
        [-0.7330, -0.7304, -3.2745],
        [-0.5274, -1.3738, -1.8533],
        [-3.4298, -2.4320, -0.1281],
        [-2.2686, -1.0497, -0.6042],
        [-2.3379, -0.2098, -2.3781],
        [-1.5278, -0.7057, -1.2405],
        [-2.2296, -0.7248, -0.8964],
        [-1.7107, -0.2361, -3.5206],
        [-2.7371, -0.1156, -3.1138],
        [-1.5082, -0.8320, -1.0685],
        [-2.4632, -1.0607, -0.5646],
        [-0.1618, -2.2198, -3.1995],
        [-0.3540, -1.6283, -2.2845],
        [-0.1197, -2.3998, -3.8145],
        [-1.2053, -0.9182, -1.2000],
        [-1.2458, -0.8397, -1.2714],
        [-2.3098, -0.6218, -1.0113],
        [-1.5131, -0.6823, -1.2934],
        [-0.2921, -1.7262, -2.5854],
        [-1.8467, -1.4833, -0.4856],
        [-0.0285, -3.7443, -5.4085],
        [-2.9426, -1.2194, -0.4279],
        [-5.9388, -4.6537, -0.0122],
        [-0.6732, -0.9300, -2.3501],
 

tensor([[-0.3695, -1.5989, -2.2366],
        [-1.7175, -0.9816, -0.8079],
        [-1.3867, -0.7471, -1.2860],
        [-0.5293, -1.3165, -1.9454],
        [-1.0668, -0.6619, -1.9657],
        [-0.3129, -1.5955, -2.7197],
        [-0.1545, -2.4423, -2.8782],
        [-1.2225, -0.9190, -1.1822],
        [-0.2807, -1.6346, -3.0014],
        [-0.5396, -1.2867, -1.9601],
        [-2.6950, -2.0978, -0.2111],
        [-1.8073, -0.6246, -1.2026],
        [-0.4958, -1.6316, -1.6332],
        [-0.5650, -1.1754, -2.0962],
        [-3.9547, -2.8005, -0.0833],
        [-0.7786, -0.9013, -2.0032],
        [-1.2118, -0.8965, -1.2231],
        [-0.0512, -3.2113, -4.6460],
        [-1.0428, -0.5259, -2.8726],
        [-5.4804, -4.2462, -0.0187],
        [-0.2074, -2.0048, -2.9451],
        [-1.3452, -1.6329, -0.6085],
        [-1.0015, -0.7696, -1.7750],
        [-1.3895, -0.6565, -1.4605],
        [-2.3394, -0.2084, -2.3887],
        [-2.0432, -1.2425, -0.5417],
        [-2.0037, -1.1561, -0.5970],
 

In [50]:
print('Accuarcy for FNN with AVERAGE word2vec embedding is =',num_correct/total)

Accuarcy for FNN with AVERAGE word2vec embedding is = 0.6658333333333334


#### b) FNN on 1st 10 words of word2vec

In [51]:
#from operator import add
fans = []
for X in df_sample['tokenize']:
    ten = []
    for i in range(10):
        res = []
        l = len(X)
        if(l==0):
            ten.append(np.zeros(3000,dtype = float))
            ten = ten[0]
            break
        #print(l)
        try:
            if(X[i] in w2v_model):
                res.append(w2v_model[X[i]])
            else:
                res.append(np.zeros(300,dtype = np.float64))
        except:
            res.append(np.zeros(300,dtype = np.float64))
        ten.extend(res[0])    
        #print(len(res))
        #print(len(ten))
    #print()
    fans.append(ten)

In [52]:
u = []
for i in fans:
    l = len(i)
    if(l not in u):
        if(l == 1):
            print(len(i[0]))
        u.append(l)

In [53]:
u

[3000]

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        fans, 
        df_sample['star_rating'],
        train_size=0.80, 
        random_state=1234)

In [55]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        current_sample = self.data[idx]
        current_target = self.targets[idx]-1
        return torch.tensor(current_sample, dtype = torch.float), torch.tensor(current_target, dtype = torch.long)

In [56]:
train_dataset = CustomDataset(data = X_train, targets = y_train.values)
test_dataset = CustomDataset(data = X_test, targets = y_test.values)

In [57]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
    num_workers=0)

In [58]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3000,100)
        self.fc2 = nn.Linear(100,10)
        self.fc3 = nn.Linear(10,3)
        self.softmax = nn.LogSoftmax(dim=1)
            
    def forward(self, features):
        x = F.relu(self.fc1(features))
        x = F.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [59]:
model = Model()
print(model)

Model(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [61]:
for epoch in range(10):
    train_loss = 0.0
    model.train()
    for data, target in train_loader:
        #print(data.shape)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss))
        

Epoch: 1 	Training Loss: 0.926934
Epoch: 2 	Training Loss: 0.852094
Epoch: 3 	Training Loss: 0.797618
Epoch: 4 	Training Loss: 0.746054
Epoch: 5 	Training Loss: 0.692199
Epoch: 6 	Training Loss: 0.653215
Epoch: 7 	Training Loss: 0.613697
Epoch: 8 	Training Loss: 0.584191
Epoch: 9 	Training Loss: 0.555301
Epoch: 10 	Training Loss: 0.526469


In [62]:
num_correct = 0
total = 0
for data, labels in test_loader:
            predictions = model(data)
            #print(predictions)
            #print(torch.argmax(predictions)==labels)
            for i in range(len(predictions)):
                if(torch.argmax(predictions[i]) == labels[i]):
                    num_correct += 1

            #print(torch.argmax(predictions[0]))
            #print(labels[0])
            #num_correct += (torch.argmax(predictions)==labels).sum()
            total += labels.size(0)

In [63]:
print('Accuarcy for FNN with 1st 10 word2vec embedding is =',num_correct/total)

Accuarcy for FNN with 1st 10 word2vec embedding is = 0.5495833333333333


df_sample.head()

### 5) Recurrent Neural Networks

In [64]:
#from operator import add
fans_rnn = []
for X in df_sample['tokenize']:
    ten = []
    for i in range(20):
        res = []
        l = len(X)
        #print(l)
        try:
            if(X[i] in w2v_model):
                res.append(w2v_model[X[i]])
            else:
                res.append(np.zeros(300,dtype = np.float64))
        except:
            res.append(np.zeros(300,dtype = np.float64))
        ten.append(res[0])    
        #print(len(res))
        #print(len(ten))
    #print()
    fans_rnn.append(ten)

In [65]:
len(fans_rnn[0][0])

300

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        fans_rnn, 
        df_sample['star_rating'],
        train_size=0.80, 
        random_state=1234)

In [67]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        current_sample = self.data[idx]
        current_target = self.targets[idx]-1
        return torch.tensor(current_sample, dtype = torch.float), torch.tensor(current_target, dtype = torch.long)

In [68]:
train_dataset = CustomDataset(data = X_train, targets = y_train.values)
test_dataset = CustomDataset(data = X_test, targets = y_test.values)

In [69]:
len(train_dataset[0][0])

C:\Users\khush\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  # Remove the CWD from sys.path while we load stuff.


20

In [70]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256,
    num_workers=0)

In [71]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

torch.Size([256, 20, 300]) torch.Size([256])


In [72]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        #self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=300, hidden_size=20, num_layers=1, batch_first=True, nonlinearity='relu')
        self.linear = nn.Linear(20, 3)

    def forward(self, X_batch):
        #embeddings = self.embedding_layer(X_batch)
        bs = X_batch.size(0)
        hidden = self.initHidden(bs)
        output, hidden = self.rnn(X_batch,hidden)
        return self.linear(output[:,-1])

    def initHidden(self,bs):
        return torch.zeros(1,bs, 20)
    
n_hidden = 20
rnn = RNN()
print(rnn)

RNN(
  (rnn): RNN(300, 20, batch_first=True)
  (linear): Linear(in_features=20, out_features=3, bias=True)
)


In [73]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

In [74]:
for epoch in range(7):
    train_loss = 0.0
    #hidden = rnn.initHidden()
    rnn.train()
    for data, target in train_loader:
        #print(len(data[0]))
        optimizer.zero_grad()
        output = rnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss))
        

Epoch: 1 	Training Loss: 1.005811
Epoch: 2 	Training Loss: 0.904948
Epoch: 3 	Training Loss: 0.870180
Epoch: 4 	Training Loss: 0.869901
Epoch: 5 	Training Loss: 0.845904
Epoch: 6 	Training Loss: 0.829127
Epoch: 7 	Training Loss: 0.817666


In [75]:
num_correct = 0
total = 0
for data, labels in test_loader:
            predictions = rnn(data)
            #print(predictions)
            #print(torch.argmax(predictions)==labels)
            for i in range(len(predictions)):
                if(torch.argmax(predictions[i]) == labels[i]):
                    num_correct += 1

            #print(torch.argmax(predictions[0]))
            #print(labels[0])
            #num_correct += (torch.argmax(predictions)==labels).sum()
            total += labels.size(0)

print('Accuarcy for RNN is =',num_correct/total)

Accuarcy for RNN is = 0.60225


### GRU

In [76]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128,
    num_workers=0)

In [77]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        #self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.gru = nn.GRU(input_size=300, hidden_size=20, num_layers=1, batch_first=True)
        self.linear = nn.Linear(20, 3)
        self.relu = nn.ReLU()

    def forward(self, X_batch):
        #embeddings = self.embedding_layer(X_batch)
        bs = X_batch.size(0)
        hidden = self.initHidden(bs)
        output, hidden = self.gru(X_batch, hidden)
        return self.linear(self.relu(output[:,-1]))

    def initHidden(self,bs):
        return torch.zeros(1,bs, 20)
    
n_hidden = 20
gru = GRU()
print(gru)

GRU(
  (gru): GRU(300, 20, batch_first=True)
  (linear): Linear(in_features=20, out_features=3, bias=True)
  (relu): ReLU()
)


In [78]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

In [79]:
for epoch in range(7):
    train_loss = 0.0
    #hidden = gru.initHidden(256)
    gru.train()
    for data, target in train_loader:
        #print(len(data[0]))
        optimizer.zero_grad()
        output = gru(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss))

Epoch: 1 	Training Loss: 0.988875
Epoch: 2 	Training Loss: 0.857992
Epoch: 3 	Training Loss: 0.817460
Epoch: 4 	Training Loss: 0.783796
Epoch: 5 	Training Loss: 0.762434
Epoch: 6 	Training Loss: 0.749043
Epoch: 7 	Training Loss: 0.738251


In [80]:
num_correct = 0
total = 0
for data, labels in test_loader:
            predictions = gru(data)
            #print(predictions)
            #print(torch.argmax(predictions)==labels)
            for i in range(len(predictions)):
                if(torch.argmax(predictions[i]) == labels[i]):
                    num_correct += 1

            #print(torch.argmax(predictions[0]))
            #print(labels[0])
            #num_correct += (torch.argmax(predictions)==labels).sum()
            total += labels.size(0)

print('Accuarcy for GRU is =',num_correct/total)

Accuarcy for GRU is = 0.6520833333333333


### LSTM

In [81]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        #self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=300, hidden_size=20, num_layers=1, batch_first=True)
        self.linear = nn.Linear(20, 3)
        self.relu = nn.ReLU()

    def forward(self, X_batch):
        #embeddings = self.embedding_layer(X_batch)
        bs = X_batch.size(0)
        hidden = self.initHidden(bs)
        output, (hidden) = self.lstm(X_batch, hidden)
        return self.linear(self.relu(output[:,-1]))

    def initHidden(self,bs):
        return (torch.zeros(1,bs, 20),torch.zeros(1,bs, 20))
    
n_hidden = 20
lstm = LSTM()
print(lstm)

LSTM(
  (lstm): LSTM(300, 20, batch_first=True)
  (linear): Linear(in_features=20, out_features=3, bias=True)
  (relu): ReLU()
)


In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)

In [83]:
for epoch in range(6):
    train_loss = 0.0
    #hidden = gru.initHidden(256)
    gru.train()
    for data, target in train_loader:
        #print(len(data[0]))
        optimizer.zero_grad()
        output = lstm(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss))

Epoch: 1 	Training Loss: 1.011960
Epoch: 2 	Training Loss: 0.856257
Epoch: 3 	Training Loss: 0.816226
Epoch: 4 	Training Loss: 0.791864
Epoch: 5 	Training Loss: 0.774722
Epoch: 6 	Training Loss: 0.761786


In [84]:
num_correct = 0
total = 0
for data, labels in test_loader:
            predictions = lstm(data)
            #print(predictions)
            #print(torch.argmax(predictions)==labels)
            for i in range(len(predictions)):
                if(torch.argmax(predictions[i]) == labels[i]):
                    num_correct += 1

            #print(torch.argmax(predictions[0]))
            #print(labels[0])
            #num_correct += (torch.argmax(predictions)==labels).sum()
            total += labels.size(0)

print('Accuarcy for LSTM is =',num_correct/total)

Accuarcy for LSTM is = 0.6438333333333334


### References

1. https://www.kaggle.com/mishra1993/pytorch-multi-layer-perceptron-mnist

2. https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

3. https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

4. https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

5. https://pytorch.org/docs/stable/generated/torch.nn.RNN.html